In [1]:
import pandas as pd

output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel(f'./file/7.page.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)
input_['SKU Count'] = input_['SKU Count'].astype(int)
input_['Page'] = input_['Page'].astype(int)

print(f'总数量：{len(input_)}')
input_['Url'].tolist()

总数量：2279


['https://www.dormanproducts.com/gsearch.aspx?type=keyword&q= Air Suspension Line Repair Kit',
 'https://www.dormanproducts.com/gsearch.aspx?type=keyword&q=110 Volt Accessory Power Outlet',
 'https://www.dormanproducts.com/gsearch.aspx?type=keyword&q=12 Volt Accessory Power Outlet Cover',
 'https://www.dormanproducts.com/gsearch.aspx?type=keyword&q=12 Volt Accessory Power Outlet Socket',
 'https://www.dormanproducts.com/gsearch.aspx?type=keyword&q=4 Wheel Drive Switch',
 'https://www.dormanproducts.com/gsearch.aspx?type=keyword&q=4WD Actuator Cable',
 'https://www.dormanproducts.com/gsearch.aspx?type=keyword&q=4WD Actuator Fork',
 'https://www.dormanproducts.com/gsearch.aspx?type=keyword&q=4WD Actuator Valve',
 'https://www.dormanproducts.com/gsearch.aspx?type=keyword&q=4WD Actuator Wiring Harness',
 'https://www.dormanproducts.com/gsearch.aspx?type=keyword&q=4WD Axle Actuator',
 'https://www.dormanproducts.com/gsearch.aspx?type=keyword&q=4WD Axle Actuator Housing',
 'https://www.dorma

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

list_url = []
while len(list_url) < input_.loc[a, 'SKU Count']:
    for i in range(input_.loc[a, 'Page']):
        b = 0
        while True:
            b += 1
            print(b, i, len(list_url))
            try:
                resp = requests.get(f'''{input_.loc[a, 'Url']}&start={i*100}&num=100''',
                                     headers=get_header(),
                                     proxies=get_proxy(),
                                     timeout=(10, 10))
                
                if resp.status_code == 200:
                    break
            except KeyboardInterrupt:
                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        soup = BeautifulSoup(resp.text, 'lxml')
        html = etree.HTML(str(soup))

        # = = = = = = = = = = = = = = =

        list_url += [f'https://www.dormanproducts.com/{href.strip()}' for href in html.xpath('//h2[@class="item-headline"]/a/@href')]
        list_url = sorted(list(set(list_url)), key=list_url.index)

        # = = = = = = = = = = = = = = =

        if len(list_url) == input_.loc[a, 'SKU Count']:
            break

resp

1 0 0


<Response [200]>

In [6]:
output = pd.DataFrame({'No': [i+1 for i in range(len(list_url))],
                       'Url': list_url})

output.to_excel('./test_url.xlsx', index=False)

output

,No,Url
0,1,https://www.dormanproducts.com/p-74380-949-006...
1,2,https://www.dormanproducts.com/p-89386-949-210...
2,3,https://www.dormanproducts.com/p-136431-949-94...
3,4,https://www.dormanproducts.com/p-9643-3600.aspx
4,5,https://www.dormanproducts.com/p-72583-963-232...
5,6,https://www.dormanproducts.com/p-100966-961-36...
6,7,https://www.dormanproducts.com/p-73284-961-364...


In [7]:
crawler_status = 'ok'

crawler_status

'ok'

In [8]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [9]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_url_error.xlsx', index=False)

output_error

""
